<a href="https://colab.research.google.com/github/AIManifest/ComfyUI-Notebook/blob/master/ComfyUI-Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#<left><font face="Monospace" size="16"><b>ComfyUI-Notebook</b></font></left>

#<left><font face="Monospace" size="6">Initialize Environment and Models</font></left>


In [ ]:
#@title <left><font face="Monospace" size="4">Step 1. Setup HOME Path for Root Working Directory (Required)</font></left>
try:
    from google.colab import drive
    gdrive = '/content/drive/'
    drive.mount(gdrive)
    HOME = '/content/'
    print(f'Root Working Directory: {HOME}')
except:
    print(f'Colab not Detected, Changing Paths to Appropriate Root Directory ("/workspace") for Jupyter')
    HOME = '/workspace/'
    print(f'Root Working Directory: {HOME}')

In [ ]:
#@title <left><font face="Monospace" size="4">Step 2: Run This Cell To Setup Jupyter Notebook/Lab to Run ComfyUI-Notebook</font></left>
import os
import subprocess

override_working_dir = False #@param{type:"boolean"}
override_dir = "/content/ComfyUI-Notebook/" #@param{type:"string"}
run_full_setup_for_jupyter = False #@param{type:"boolean"}
install_comfy_notebook = True #@param{type:"boolean"}
install_comfy_ui = False #@param{type:"boolean"}
install_esrgan = False #@param{type:"boolean"}
xformers_version = 'xformers==0.0.21.dev583' #@param{type:"string"}

def setup_comfy():
    if run_full_setup_for_jupyter:
        subprocess.run('curl -fsSL https://deb.nodesource.com/setup_18.x | sudo -E bash -', shell=True)
        subprocess.run(['sudo', 'apt-get', 'install', '-y', 'nodejs'])

        subprocess.run(['apt-get', 'update'])
        subprocess.run(['apt-get', 'install', 'ffmpeg', 'libsm6', 'libxext6', '-y'])
        subprocess.run(['pip', 'install', 'pandas'])

        subprocess.run(['sudo', 'apt-get', 'update'])
        subprocess.run(['sudo', 'apt-get', 'install', 'npm', '-y'])
    if install_comfy_notebook:
        os.system("git clone https://github.com/AIManifest/ComfyUI-Notebook.git")
    if install_comfy_ui:
        os.system("git clone https://github.com/comfyanonymous/ComfyUI.git")
    if install_esrgan:
        os.system("git clone https://github.com/AIManifest/ESRGAN.git")

    WORKING_DIR = os.path.join(HOME, "ComfyUI-Notebook/")
    if override_working_dir:
        WORKING_DIR = override_dir
    os.chdir(WORKING_DIR)
    print(f'Running in Directory: {WORKING_DIR}')
    subprocess.run(['pip', 'install', '-r', 'requirements.txt'])
    subprocess.Popen(['pip', 'install', str(xformers_version), 'pytorch-lightning', 'flash-attn', '--no-build-isolation'])

setup_comfy()


In [ ]:
#@title <left><font face="Monospace" size="4">Step 3.a: Run This Cell To Copy Models from One Folder to Root Folder</font></left>
source_folder = "/content/drive/MyDrive/ComfyUI/models/checkpoints" #@param{type:"string"}
root_folder = "/content/ComfyUI-Notebook/models" #@param{type:"string"}
!cp -r "$source_folder" "$root_folder" --verbose

In [ ]:
#@title <left><font face="Monospace" size="4">Step 3.b: Install Models</font></left>
import os
import time
import shutil
import requests
from tqdm.auto import tqdm

model_dir = "/content/ComfyUI-Notebook/models/checkpoints/" #@param{type:"string"}
vae_dir = "/content/ComfyUI/models/vae/" #@param{type:"string"}
lora_dir = "/content/ComfyUI/models/loras/" #@param{type:"string"}
controlnet_dir = "/content/ComfyUI-Notebook/models/controlnet" #@param{type:"string"}

# Add boolean parameters for each model
download_sd_xl_offset_example_lora_1_0 = False #@param {type:"boolean"}
download_sd_xl_base_1_0 = False #@param {type:"boolean"}
download_sd_xl_refiner_1_0 = False #@param {type:"boolean"}
download_sd_xl_base_1_0_0_9vae = True #@param {type:"boolean"}
download_sd_xl_refiner_1_0_0_9vae = False #@param {type:"boolean"}
download_dreamshaperXL10_alpha2XI10 = False #@param {type:"boolean"}
download_psyanimated_xl = False #@param{type:"boolean"}
download_crystalClearxl_ccxl = False #@param{type:"boolean"}
download_canny_sdxl = True #@param{type:"boolean"}
download_depth_sdxl = True #@param{type:"boolean"}

def download_sdxl():
    models_to_download = [
        ("https://huggingface.co/diffusers/controlnet-depth-sdxl-1.0/resolve/main/diffusion_pytorch_model.safetensors", controlnet_dir, "sdxl_1-0_depth.safetensors", download_depth_sdxl),
        ("https://huggingface.co/diffusers/controlnet-canny-sdxl-1.0/resolve/main/diffusion_pytorch_model.safetensors", controlnet_dir, "sdxl_1-0_canny.safetensors", download_canny_sdxl),
        ("https://civitai.com/api/download/models/133832", model_dir, "crystalClearxl_ccxl.safetensors", download_crystalClearxl_ccxl),
        ("https://civitai.com/api/download/models/136271", model_dir, "psyanimated_xl.safetensors", download_psyanimated_xl),
        ("https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_offset_example-lora_1.0.safetensors", lora_dir, "sd_xl_offset_example-lora_1.0.safetensors", download_sd_xl_offset_example_lora_1_0),
        ("https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_base_1.0.safetensors", model_dir, "sd_xl_base_1.0.safetensors", download_sd_xl_base_1_0),
        ("https://huggingface.co/stabilityai/stable-diffusion-xl-refiner-1.0/resolve/main/sd_xl_refiner_1.0.safetensors", model_dir, "sd_xl_refiner_1.0.safetensors", download_sd_xl_refiner_1_0),
        ("https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_base_1.0_0.9vae.safetensors", model_dir, "sd_xl_base_1.0_0.9vae.safetensors", download_sd_xl_base_1_0_0_9vae),
        ("https://huggingface.co/stabilityai/stable-diffusion-xl-refiner-1.0/resolve/main/sd_xl_refiner_1.0_0.9vae.safetensors", model_dir, "sd_xl_refiner_1.0_0.9vae.safetensors", download_sd_xl_refiner_1_0_0_9vae),
        ("https://civitai.com/api/download/models/126688", model_dir, "dreamshaperXL10_alpha2XI10.safetensors", download_dreamshaperXL10_alpha2XI10)
    ]

    username = "AIManifest"
    password = "winterfellCCJP04"

    for model_URL, folder_path, filename, download in models_to_download:
        if not download:
            continue

        if not os.path.exists(folder_path):
            os.makedirs(folder_path, exist_ok=True)

        if not os.path.exists(os.path.join(folder_path, filename)):
            start_time = time.time()

            try:
                auth = (username, password) if username and password else None
                with requests.get(model_URL, stream=True, auth=auth) as r:

                    total_length = int(r.headers.get("Content-Length"))

                    with tqdm.wrapattr(r.raw, "read", total=total_length, desc="..Downloading..", colour='orange') as raw:

                        with open(os.path.join(folder_path, filename), 'wb')as output:
                            shutil.copyfileobj(raw, output)

                end_time = time.time()
                final_time = end_time-start_time
                print(f"\033[38;2;255;192;203mDone! Model {filename} downloaded in {final_time:.2f} seconds!")
                print(f"\033[38;2;255;192;203mFile {filename} downloaded and saved to {folder_path}")
            except Exception as e:
                print(f"An error occurred while downloading the model: {e}")

download_sdxl()

#<left><font face="Monospace" size="6">Run ComfyUI-Notebook For Web Browser</font></left>

In [ ]:
#@title <left><font face="Monospace" size="4">Run ComfyUI-Notebook For Web Browser</font></left>
working_dir = "/content/ComfyUI" #@param{type:"string"}
%cd $working_dir
!npm install -g localtunnel

import subprocess
import threading
import time
import socket
import urllib.request

preview_method = "auto" #@param{type:"string"}["none", "auto", "latent2rgb", "taesd"]

def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI-Notebook finished loading, trying to launch localtunnel (if it gets stuck here localtunnel is having issues)\n")

  print("The password/enpoint ip for localtunnel is:", urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))
  p = subprocess.Popen(["lt", "--port", "{}".format(port)], stdout=subprocess.PIPE)
  for line in p.stdout:
    print(line.decode(), end='')


threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server --preview-method $preview_method



In [ ]:
#@title <left><font face="Monospace" size="4">(Advanced) Run ComfyUI-Notebook For Web Browser</font></left>

#@markdown <left><font face="Monospace" size="4">Open This Cell to Add Your Extra ComfyUI Launch Arguments</font></left>

working_dir = "/content/ComfyUI" #@param{type:"string"}
%cd $working_dir
!npm install -g localtunnel

import subprocess
import threading
import time
import socket
import urllib.request

# preview_method = "auto" #@param{type:"string"}["none", "auto", "latent2rgb", "taesd"]
# listen = "127.0.0.1" #@param{type:"string"}
# port = 8188 #@param{type:"number"}
# enable_cors_header = None #@param{type:"string"}
# extra_model_paths_config = None #@param{type:"string"}
# output_directory = None #@param{type:"string"}
# temp_directory = None #@param{type:"string"}
# auto_launch = False #@param{type:"boolean"}

# disable_auto_launch = False #@param{type:"boolean"}

# cuda_device = 0 #@param{type:"number"}

# cuda_malloc = False #@param{type:"boolean"}
# disable_cuda_malloc = False #@param{type:"boolean"}

# dont_upcast_attention = False #@param{type:"boolean"}
# force_fp32 = False #@param{type:"boolean"}
# force_fp16 = False #@param{type:"boolean"}

# fp16_vae = False #@param{type:"boolean"}
# bf16_vae = False #@param{type:"boolean"}

# directml = None #@param{type:"number"}
# use_split_cross_attention = False #@param{type:"boolean"}
# use_quad_cross_attention = False #@param{type:"boolean"}
# use_pytorch_cross_attention = False #@param{type:"boolean"}
# disable_xformers = False #@param{type:"boolean"}

# gpu_only = False #@param{type:"boolean"}
# highvram = False #@param{type:"boolean"}
# normalvram = False #@param{type:"boolean"}
# lowvram = False #@param{type:"boolean"}
# novram = False #@param{type:"boolean"}
# cpu = False #@param{type:"boolean"}

# dont_print_server = False #@param{type:"boolean"}
# quick_test_for_ci = False #@param{type:"boolean"}
# windows_standalone_build = False #@param{type:"boolean"}

# disable_metadata = False #@param{type:"boolean"}


def iframe_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nComfyUI-Notebook finished loading, trying to launch localtunnel (if it gets stuck here localtunnel is having issues)\n")

  print("The password/enpoint ip for localtunnel is:", urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))
  p = subprocess.Popen(["lt", "--port", "{}".format(port)], stdout=subprocess.PIPE)
  for line in p.stdout:
    print(line.decode(), end='')


threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()

!python main.py --dont-print-server \
--preview-method $preview_method \
# --listen $listen \
# --port $port\
# --enable-cors-header
# --extra-model-paths-config
# --output-directory
# --temp-directory
# --auto-launch
# --disable-auto-launch
# --cuda-device
# --cuda-malloc
# --disable-cuda-malloc
# --dont-upcast-attention
# --force-fp32 \
# --force-fp16
# --fp16-vae
# --bf16-vae
# --directml
# --disable-xformers
# --gpu-only
# --highvram
# --normalvram
# --lowvram
# --novram
# --cpu
# --quick-test-for-ci
# --windows-standalone-build
# --disable-metadata

#<left><font face="Monospace" size="6">Run ComfyUI-Notebook System</font></left>

##<left><font face="Monospace" size="6">Run SDXL -> Single Prompt</font></left>


In [ ]:
#@title <left><font face="Monospace" size="4">Set CudaMallocAsync</font></left>
import os
env_var = "backend:cudaMallocAsync"
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = env_var
os.environ['PYTORCH_CUDA_ALLOC_CONF']

In [ ]:
#@title <left><font face="Monospace" size="4">RUN SDXL -> Single Prompt!</font></left>
working_dir = "/content/ComfyUI-Notebook" #@param{type:"string"}
%cd $working_dir
import gc
import torch
import random
import nodes
import sdxlrun
import ipywidgets
from types import SimpleNamespace
from comfy import model_management
from sdxlrun import runsdxl, loadsdxl
from pytorch_lightning import seed_everything
from refinersdxlrun import runsdxlrefiner, loadsdxlrefiner
from iprogress import iprogress

def SDXL_ARGS():
    prompt = "a photorealistic dragon, very very photorealistic, trippy cartoon style with thick outlines" #@param{type:"string"}
    negativeprompt = "(woman:1.5), girl, lady, female, bad art, blurry, blurry background, bokeh, bad drawing, poor composition, poor use of textures, poor use of colors, poor art drawing, portrait, ugly, face shot, Overbaked, over contrasted, extra fingers, deformed hands, over saturated, ((3d, cartoon, anime, sketches)), (worst quality:2), (low quality:2), (normal quality:2), lowres, normal quality, ((monochrome)), ((grayscale)), bad anatomy, out of view, cut off, ugly, deformed, mutated, ((young)), EasyNegative, paintings, sketches, (worst quality:2), (low quality:2), (normal quality:2), lowres, normal quality, ((monochrome)), ((grayscale)), skin spots, acnes, skin blemishes, age spot, glans,extra fingers,fewer fingers, (ugly eyes, deformed iris, deformed pupils, fused lips and teeth:1.2), (un-detailed skin, semi-realistic, cgi, 3d, render, sketch, cartoon, drawing, anime:1.2), text, close up, cropped, out of frame, worst quality, low quality, jpeg artifacts, ugly, duplicate, morbid, mutilated, extra fingers, mutated hands, poorly drawn hands, poorly drawn face, mutation, deformed, blurry, dehydrated, bad anatomy, bad proportions, extra limbs, cloned face, disfigured, gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs, fused fingers, too many fingers, long neck," #@param{type:"string"}
    #@markdown ${\textsf{Base Model Options}}$
    ckpt_path='/content/ComfyUI/models/checkpoints/dreamshaperXL10_alpha2XI10.safetensors'
    ckpt_name='sd_xl_base_1.0_0.9vae.safetensors' #@param{type:"string"}["crystalClearxl_ccxl.safetensors", "psyanimated_xl.safetensors", "sd_xl_base_1.0_0.9vae.safetensors", "dreamshaperXL10_alpha2XI10.safetensors"]
    lora_name = "None"#@param{type:"string"}["None", "sd_xl_offset_example-lora_1.0.safetensors", "greg_rutkowski_xl_2.safetensors"]
    if lora_name == "None":
        lora_name = None
    else:
        strength_model = 0.5 #@param{type:"number"}
        strength_clip = 0.5 #@param{type:"number"}

    #@markdown ${\textsf{Base Model}}$
    steps = 60 #@param{type:"number"}
    start_step = 0 #@param{type:"number"}
    last_step = 55 #@param{type:"number"}

    #refiner model
    use_refiner =  False #@param{type:"boolean"}
    last_step = last_step if use_refiner else steps
    refiner_steps = steps
    refiner_start_step = last_step
    refiner_last_step = steps

    #@markdown ${\textsf{Image-Batch Settings}}$
    imagewidth = 768 #@param{type:"number"}
    imageheight = 1216   #@param{type:"number"}
    batch_size = 1 #@param{type:"number"}

    cfg = 3.5 #@param{type:"number"}
    seed = -1 #@param{type:"number"}
    if seed == -1:
        seed = random.randint(1, 432043209)
    seed = seed_everything(seed)

    #@markdown ${\textsf{Sampler Settings}}$
    sampler_name = "dpmpp_2m_sde_gpu" #@param{type:"string"} ["euler", "euler_ancestral", "heun", "dpm_2", "dpm_2_ancestral", "lms", "dpm_fast", "dpm_adaptive", "dpmpp_2s_ancestral", "dpmpp_sde", "dpmpp_sde_gpu", "dpmpp_2m", "dpmpp_2m_sde", "dpmpp_2m_sde_gpu", "ddim", "uni_pc", "uni_pc_bh2"]
    scheduler = "exponential" #@param{type:"string"} ["normal", "karras", "exponential", "simple", "ddim_uniform"]

    #@markdown ${\textsf{Sampler Options}}$
    s_churn = 0.01 # @param {type:"slider", min:0, max:1, step:0.01}
    s_tmin = 0.01 # @param {type:"slider", min:0, max:1, step:0.01}
    s_tmax = float('inf')
    s_noise = 1. # @param {type:"slider", min:0, max:1, step:0.01}

    eta = 1. #@param{type:"number"}

    order = 4 #@param{type:"slider", min:1, max:4, step:1}

    #@markdown ${\textsf{CLIP Options}}$
    stop_at_last_layer = -6 #@param{type:"number"}
    width = 5120 #@param{type:"number"}
    height = 5120 #@param{type:"number"}
    crop_w = 0 #@param{type:"number"}
    crop_h = 0 #@param{type:"number"}
    target_width = 5120 #@param{type:"number"}
    target_height = 5120 #@param{type:"number"}

    #@markdown ${\textsf{Noise Options}}$
    denoise = 1.0 #@param{type:"number"}
    force_full_denoise = False #@param{type:"boolean"}
    disable_noise = False #@param{type:"boolean"}
    force_full_denoise = force_full_denoise if use_refiner else True
    noisedevice = "cuda:0" #@param{type:"string"}["cuda:0", "cpu"]

    #@markdown ${\textsf{REFINER OPTIONS}}$
    refinerckpt_name='sd_xl_refiner_1.0_0.9vae.safetensors' #@param{type:"string"} ["sd_xl_refiner_1.0_0.9vae.safetensors", "sd_xl_refiner_1.0.safetensors"]
    refinerforce_full_denoise = True #@param{type:"boolean"}
    refinerdisable_noise = True #@param{type:"boolean"}
    refinerwidth = 4096 #@param{type:"number"}
    refinerheight = 4096 #@param{type:"number"}
    ascore = 3.0 #@param{type:"number"}

    #@markdown ${\textsf{Controlnet Canny OPTIONS}}$
    is_controlnet = True #@param{type:"boolean"}
    controlnet_name = "sdxl_1-0_depth.safetensors" #@param{type:"string"} ["sdxl_1-0_depth.safetensors", "sdxl_1-0_canny.safetensors"]
    controlnet_image = "/content/upscaled_dragon.png" #@param{type:"string"}
    controlnet_low_threshold = 0.19 # @param {type:"slider", min:0.01, max:0.99, step:0.01}
    controlnet_high_threshold = 0.87 # @param {type:"slider", min:0.01, max:0.99, step:0.01}
    controlnet_strength = 0.67 # @param {type:"slider", min:0, max:10, step:0.01}
    controlnet_start_percent = 0 # @param {type:"slider", min:0, max:1, step:0.001}
    controlnet_end_percent = 1.0 # @param {type:"slider", min:0, max:1, step:0.001}

    #@markdown ${\textsf{Save Options}}$
    output_folder = "civit" #@param{type:"string"}
    saveprefix = f'{sampler_name}_{scheduler}'
    save_base_image = True #@param{type:"boolean"}

    vae_decode_method = "tiled" #@param{type:"string"}["normal", "tiled"]

    use_preview = True #@param{type:"boolean"}
    disable_metadata = False #@param{type:"boolean"}

    return locals()

sdxl_args = SDXL_ARGS()
sdxl_args = SimpleNamespace(**sdxl_args)

load_settings_metadata = False #@param{type:"boolean"}
if load_settings_metadata:
    import json
    from PIL import Image as pilimage
    load_settings_from_image = '/content/ComfyUI-Notebook/civit/euler_ancestral_exponential_00002_.png' #@param{type:"string"}
    png_image = pilimage.open(load_settings_from_image)
    pnginfo = png_image.info

    for key in sdxl_args.__dict__.keys():
        if key in pnginfo:
            value = json.loads(pnginfo[key])
            print(f'Adding value to key: {value}')
            setattr(sdxl_args, key, value)

with torch.inference_mode():
    try:
        out
        if sdxl_args.ckpt_name != checkpoint:
            del out
            del model
            del samples
            gc.collect()
            torch.cuda.empty_cache()
            out = loadsdxl(sdxl_args)
            checkpoint = sdxl_args.ckpt_name
        if sdxl_args.use_refiner:
            refined_out
        if sdxl_args.is_controlnet:
            control_net
    except:
        out = loadsdxl(sdxl_args)
        checkpoint = sdxl_args.ckpt_name #create checkpoint to trigger loading a new checkpoint
        if sdxl_args.use_refiner:
            refined_out = loadsdxlrefiner(sdxl_args)
        if sdxl_args.is_controlnet:
            controlnet_loader = nodes.ControlNetLoader()
            control_net = controlnet_loader.load_controlnet(sdxl_args.controlnet_name)

    print('Running with settings:')
    print(f'Prompt: {sdxl_args.prompt}')
    print(f'Negative Prompt: {sdxl_args.negativeprompt}')
    for key, value in sdxl_args.__dict__.items():
        if not "prompt" in key and not "negativeprompt" in key:
            print(f'{key}: {value}')
    model, samples = runsdxl(sdxl_args, out, control_net)
    model_management.unload_model(model)
    model_management.unload_model(out[1].patcher)
    sdxlrun.get_device_memory()
    if sdxl_args.use_refiner:
        refinermodel, refinedsamples = runsdxlrefiner(sdxl_args, samples, model, refined_out)
        model_management.unload_model(refinermodel)
        model_management.unload_model(refined_out[1].patcher)
        sdxlrun.get_device_memory()

torch.cuda.empty_cache()
gc.collect()
checkpoint = sdxl_args.ckpt_name

##<left><font face="Monospace" size="6">RUN SDXL -> Single Prompt for All Schedulers and Samplers</font></left>

In [ ]:
#@title <left><font face="Monospace" size="4">Set CudaMallocAsync</font></left>
import os
env_var = "backend:cudaMallocAsync"
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = env_var
os.environ['PYTORCH_CUDA_ALLOC_CONF']

In [ ]:
#@title $ {\textsf {RUN SDXL -> Single Prompt for All Schedulers and Samplers}}$
working_dir = "/content/ComfyUI-Notebook" #@param{type:"string"}
%cd $working_dir
import gc
import torch
import random
import sdxlrun
from types import SimpleNamespace
from comfy import model_management
from sdxlrun import runsdxl, loadsdxl
from pytorch_lightning import seed_everything
from refinersdxlrun import runsdxlrefiner, loadsdxlrefiner
from iprogress import iprogress

def SDXL_ARGS():
    prompt = "emphasizing every detail, hyperrealism, ambient lighting, ray tracing, amazing detail, full body photo, Hyperdetailed masterpiece, soft dramatic lighting, (extreme detail:1.2),  best quality, (photorealistic:1.4) photo of sun wu kong, (monkey king), kung-fu stance, holding a large golden pole,  detailed face, cinematic background, cinematic light, (masterpiece, best quality:1.5)" #@param{type:"string"}
    negativeprompt = "bad art, blurry, blurry background, bad drawing, poor composition, poor use of textures, poor use of colors, poor art drawing, portrait, ugly, face shot, Overbaked, over contrasted, over saturated, ((3d, cartoon, anime, sketches)), (worst quality:2), (low quality:2), (normal quality:2), lowres, normal quality, ((monochrome)), ((grayscale)), bad anatomy, out of view, cut off, ugly, deformed, mutated, ((young)), EasyNegative, paintings, sketches, (worst quality:2), (low quality:2), (normal quality:2), lowres, normal quality, ((monochrome)), ((grayscale)), skin spots, acnes, skin blemishes, age spot, glans,extra fingers,fewer fingers, (ugly eyes, deformed iris, deformed pupils, fused lips and teeth:1.2), (un-detailed skin, semi-realistic, cgi, 3d, render, sketch, cartoon, drawing, anime:1.2), text, close up, cropped, out of frame, worst quality, low quality, jpeg artifacts, ugly, duplicate, morbid, mutilated, extra fingers, mutated hands, poorly drawn hands, poorly drawn face, mutation, deformed, blurry, dehydrated, bad anatomy, bad proportions, extra limbs, cloned face, disfigured, gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs, fused fingers, too many fingers, long neck," #@param{type:"string"}
    #@markdown ${\textsf{Base Model Options}}$
    ckpt_path='/content/ComfyUI/models/checkpoints/dreamshaperXL10_alpha2XI10.safetensors'
    ckpt_name='crystalClearxl_ccxl.safetensors' #@param{type:"string"}["crystalClearxl_ccxl.safetensors", "psyanimated_xl.safetensors", "sd_xl_base_1.0_0.9vae.safetensors", "dreamshaperXL10_alpha2XI10.safetensors"]
    lora_name = "None"#@param{type:"string"}["None", "sd_xl_offset_example-lora_1.0.safetensors", "greg_rutkowski_xl_2.safetensors"]
    if lora_name == "None":
        lora_name = None
    else:
        strength_model = 0.5 #@param{type:"number"}
        strength_clip = 0.5 #@param{type:"number"}

    #@markdown ${\textsf{Base Model}}$
    steps = 40 #@param{type:"number"}
    start_step = 0 #@param{type:"number"}
    last_step = 35 #@param{type:"number"}

    #refiner model
    use_refiner =  False #@param{type:"boolean"}
    last_step = last_step if use_refiner else steps
    refiner_steps = steps
    refiner_start_step = last_step
    refiner_last_step = steps

    #@markdown ${\textsf{Image-Batch Settings}}$
    imagewidth = 896 #@param{type:"number"}
    imageheight = 1264   #@param{type:"number"}
    batch_size = 1 #@param{type:"number"}

    cfg = 3 #@param{type:"number"}
    seed = -1 #@param{type:"number"}
    if seed == -1:
        seed = random.randint(1, 432043209)
    seed = seed_everything(seed)

    #@markdown ${\textsf{Sampler Settings}}$
    sampler_name = "euler_ancestral" #@param{type:"string"} ["euler", "euler_ancestral", "heun", "dpm_2", "dpm_2_ancestral", "lms", "dpm_fast", "dpm_adaptive", "dpmpp_2s_ancestral", "dpmpp_sde", "dpmpp_sde_gpu", "dpmpp_2m", "dpmpp_2m_sde", "dpmpp_2m_sde_gpu", "ddim", "uni_pc", "uni_pc_bh2"]
    scheduler = "exponential" #@param{type:"string"} ["normal", "karras", "exponential", "simple", "ddim_uniform"]

    #@markdown ${\textsf{Sampler Options}}$
    s_churn = 0. # @param {type:"slider", min:0, max:1, step:0.01}
    s_tmin = 0.1 # @param {type:"slider", min:0, max:1, step:0.01}
    s_tmax = float('inf')
    s_noise = 1. # @param {type:"slider", min:0, max:1, step:0.01}

    eta = 1. #@param{type:"number"}

    order = 4 #@param{type:"slider", min:1, max:4, step:1}

    #@markdown ${\textsf{CLIP Options}}$
    stop_at_last_layer = None #@param{type:"number"}
    width = 4096 #@param{type:"number"}
    height = 4096 #@param{type:"number"}
    crop_w = 0 #@param{type:"number"}
    crop_h = 0 #@param{type:"number"}
    target_width = 4096 #@param{type:"number"}
    target_height = 4096 #@param{type:"number"}

    #@markdown ${\textsf{Noise Options}}$
    denoise = 1.0 #@param{type:"number"}
    force_full_denoise = False #@param{type:"boolean"}
    disable_noise = False #@param{type:"boolean"}
    force_full_denoise = force_full_denoise if use_refiner else True
    noisedevice = "cuda:0" #@param{type:"string"}["cuda:0", "cpu"]

    #@markdown ${\textsf{REFINER OPTIONS}}$
    refinerckpt_name='sd_xl_refiner_1.0_0.9vae.safetensors' #@param{type:"string"} ["sd_xl_refiner_1.0_0.9vae.safetensors", "sd_xl_refiner_1.0.safetensors"]
    refinerforce_full_denoise = True #@param{type:"boolean"}
    refinerdisable_noise = True #@param{type:"boolean"}
    refinerwidth = 4096 #@param{type:"number"}
    refinerheight = 4096 #@param{type:"number"}
    ascore = 3.0 #@param{type:"number"}

    #@markdown ${\textsf{Save Options}}$
    output_folder = "civit" #@param{type:"string"}
    saveprefix = f'{sampler_name}_{scheduler}'
    save_base_image = False #@param{type:"boolean"}

    vae_decode_method = "tiled" #@param{type:"string"}["normal", "tiled"]

    use_preview = True #@param{type:"boolean"}
    disable_metadata = False #@param{type:"boolean"}

    return locals()

# ["euler", "euler_ancestral", "heun", "dpm_2", "dpm_2_ancestral", "lms", "dpm_fast", "dpm_adaptive",]
sampler_names = ["euler", "euler_ancestral", "heun", "dpm_2", "dpm_2_ancestral", "lms", "dpm_fast", "dpm_adaptive", "dpmpp_2s_ancestral", "dpmpp_sde", "dpmpp_sde_gpu", "dpmpp_2m", "dpmpp_2m_sde", "dpmpp_2m_sde_gpu", "ddim", "uni_pc", "uni_pc_bh2"]
schedulers = ["normal", "karras", "exponential", "simple", "ddim_uniform"]
randomize_seed = False #@param{type:"boolean"}

for sampler_name in iprogress(sampler_names, colour="synthlite"):
    for scheduler in iprogress(schedulers, colour="cotton_candy"):
        if randomize_seed:
            seed = seed_everything(random.randint(2 ** 64))
        sdxl_args = SDXL_ARGS()
        sdxl_args = SimpleNamespace(**sdxl_args)
        sdxl_args.sampler_name = sampler_name
        sdxl_args.scheduler = scheduler
        sdxl_args.saveprefix = f'{sampler_name}_{scheduler}'
        with torch.inference_mode():
            try:
                out
                if sdxl_args.ckpt_name != checkpoint:
                    del out
                    del model
                    del samples
                    gc.collect()
                    torch.cuda.empty_cache()
                    out = loadsdxl(sdxl_args)
                    checkpoint = sdxl_args.ckpt_name
                if sdxl_args.use_refiner:
                    refined_out
            except:
                out = loadsdxl(sdxl_args)
                checkpoint = sdxl_args.ckpt_name #create checkpoint to trigger loading a new checkpoint
                if sdxl_args.use_refiner:
                    refined_out = loadsdxlrefiner(sdxl_args)

            print('Running with settings:')
            print(f'Prompt: {sdxl_args.prompt}')
            print(f'Negative Prompt: {sdxl_args.negativeprompt}')
            for key, value in sdxl_args.__dict__.items():
                if not "prompt" in key and not "negativeprompt" in key:
                    print(f'{key}: {value}')
            model, samples = runsdxl(sdxl_args, out)
            model_management.unload_model(model)
            model_management.unload_model(out[1].patcher)
            sdxlrun.get_device_memory()
            if sdxl_args.use_refiner:
                refinermodel, refinedsamples = runsdxlrefiner(sdxl_args, samples, model, refined_out)
                model_management.unload_model(refinermodel)
                model_management.unload_model(refined_out[1].patcher)
                sdxlrun.get_device_memory()

        torch.cuda.empty_cache()
        gc.collect()
        checkpoint = sdxl_args.ckpt_name

##<left><font face="Monospace" size="6">Run SDXL -> Multi-Prompt</font></left>

In [ ]:
#@title <left><font face="Monospace" size="4">Set CudaMallocAsync</font></left>
import os
env_var = "backend:cudaMallocAsync"
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = env_var
os.environ['PYTORCH_CUDA_ALLOC_CONF']

In [ ]:
prompts = {
    1: "(horror), (insidious), haunted hotel, ghostly apparitions, (dark colors), (eerie), high quality, dim light, spooky atmosphere",
    2: "(horror), (insidious), cursed graveyard, undead rising, (dark colors), (eerie), high quality, dim light, spooky atmosphere",
    3: "(horror), (insidious), demonic possession, exorcism gone wrong, (dark colors), (eerie), high quality, dim light, spooky atmosphere",
    4: "(horror), (insidious), abandoned hospital, ghostly patients, (dark colors), (eerie), high quality, dim light, spooky atmosphere",
    5: "(horror), (insidious), supernatural portal, otherworldly creatures, (dark colors), (eerie), high quality, dim light, spooky atmosphere",
    6: "(horror), (insidious), haunted dollhouse, miniature horrors, (dark colors), (eerie), high quality, dim light, spooky atmosphere",
    7: "(horror), (insidious), vengeful spirit , revenge from beyond the grave , (dark colors) , (eerie) , high quality , dim light , spooky atmosphere",
    8: "(horror) , (insidious) , cursed painting , trapped souls , (dark colors) , (eerie) , high quality , dim light , spooky atmosphere",
    9: "(horror) , (insidious) , haunted mirror , alternate dimension , (dark colors) , (eerie) , high quality , dim light , spooky atmosphere",
    10: "(horror) , (insidious) , supernatural storm , ghostly shipwreck , (dark colors) , (eerie) , high quality , dim light , spooky atmosphere"
}

negative_prompts = {
    1: "bad art, blurry, blurry background, bad drawing, poor composition, poor use of textures, poor use of colors, poor art drawing, portrait, ugly, face shot, Overbaked, over contrasted, over saturated, ((3d, cartoon, anime, sketches)), (worst quality:2), (low quality:2), (normal quality:2), lowres, normal quality, ((monochrome)), ((grayscale)), bad anatomy, out of view, cut off, ugly, deformed, mutated, ((young)), EasyNegative, paintings, sketches, (worst quality:2), (low quality:2), (normal quality:2), lowres, normal quality, ((monochrome)), ((grayscale)), skin spots, acnes, skin blemishes, age spot, glans,extra fingers,fewer fingers, (ugly eyes, deformed iris, deformed pupils, fused lips and teeth:1.2), (un-detailed skin, semi-realistic, cgi, 3d, render, sketch, cartoon, drawing, anime:1.2), text, close up, cropped, out of frame, worst quality, low quality, jpeg artifacts, ugly, duplicate, morbid, mutilated, extra fingers, mutated hands, poorly drawn hands, poorly drawn face, mutation, deformed, blurry, dehydrated, bad anatomy, bad proportions, extra limbs, cloned face, disfigured, gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs, fused fingers, too many fingers, long neck,",
    2: "bad art, blurry, blurry background, bad drawing, poor composition, poor use of textures, poor use of colors, poor art drawing, portrait, ugly, face shot, Overbaked, over contrasted, over saturated, ((3d, cartoon, anime, sketches)), (worst quality:2), (low quality:2), (normal quality:2), lowres, normal quality, ((monochrome)), ((grayscale)), bad anatomy, out of view, cut off, ugly, deformed, mutated, ((young)), EasyNegative, paintings, sketches, (worst quality:2), (low quality:2), (normal quality:2), lowres, normal quality, ((monochrome)), ((grayscale)), skin spots, acnes, skin blemishes, age spot, glans,extra fingers,fewer fingers, (ugly eyes, deformed iris, deformed pupils, fused lips and teeth:1.2), (un-detailed skin, semi-realistic, cgi, 3d, render, sketch, cartoon, drawing, anime:1.2), text, close up, cropped, out of frame, worst quality, low quality, jpeg artifacts, ugly, duplicate, morbid, mutilated, extra fingers, mutated hands, poorly drawn hands, poorly drawn face, mutation, deformed, blurry, dehydrated, bad anatomy, bad proportions, extra limbs, cloned face, disfigured, gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs, fused fingers, too many fingers, long neck,",
    3: "bad art, blurry, blurry background, bad drawing, poor composition, poor use of textures, poor use of colors, poor art drawing, portrait, ugly, face shot, Overbaked, over contrasted, over saturated, ((3d, cartoon, anime, sketches)), (worst quality:2), (low quality:2), (normal quality:2), lowres, normal quality, ((monochrome)), ((grayscale)), bad anatomy, out of view, cut off, ugly, deformed, mutated, ((young)), EasyNegative, paintings, sketches, (worst quality:2), (low quality:2), (normal quality:2), lowres, normal quality, ((monochrome)), ((grayscale)), skin spots, acnes, skin blemishes, age spot, glans,extra fingers,fewer fingers, (ugly eyes, deformed iris, deformed pupils, fused lips and teeth:1.2), (un-detailed skin, semi-realistic, cgi, 3d, render, sketch, cartoon, drawing, anime:1.2), text, close up, cropped, out of frame, worst quality, low quality, jpeg artifacts, ugly, duplicate, morbid, mutilated, extra fingers, mutated hands, poorly drawn hands, poorly drawn face, mutation, deformed, blurry, dehydrated, bad anatomy, bad proportions, extra limbs, cloned face, disfigured, gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs, fused fingers, too many fingers, long neck,",
    4: "bad art, blurry, blurry background, bad drawing, poor composition, poor use of textures, poor use of colors, poor art drawing, portrait, ugly, face shot, Overbaked, over contrasted, over saturated, ((3d, cartoon, anime, sketches)), (worst quality:2), (low quality:2), (normal quality:2), lowres, normal quality, ((monochrome)), ((grayscale)), bad anatomy, out of view, cut off, ugly, deformed, mutated, ((young)), EasyNegative, paintings, sketches, (worst quality:2), (low quality:2), (normal quality:2), lowres, normal quality, ((monochrome)), ((grayscale)), skin spots, acnes, skin blemishes, age spot, glans,extra fingers,fewer fingers, (ugly eyes, deformed iris, deformed pupils, fused lips and teeth:1.2), (un-detailed skin, semi-realistic, cgi, 3d, render, sketch, cartoon, drawing, anime:1.2), text, close up, cropped, out of frame, worst quality, low quality, jpeg artifacts, ugly, duplicate, morbid, mutilated, extra fingers, mutated hands, poorly drawn hands, poorly drawn face, mutation, deformed, blurry, dehydrated, bad anatomy, bad proportions, extra limbs, cloned face, disfigured, gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs, fused fingers, too many fingers, long neck,",
    5: "bad art, blurry, blurry background, bad drawing, poor composition, poor use of textures, poor use of colors, poor art drawing, portrait, ugly, face shot, Overbaked, over contrasted, over saturated, ((3d, cartoon, anime, sketches)), (worst quality:2), (low quality:2), (normal quality:2), lowres, normal quality, ((monochrome)), ((grayscale)), bad anatomy, out of view, cut off, ugly, deformed, mutated, ((young)), EasyNegative, paintings, sketches, (worst quality:2), (low quality:2), (normal quality:2), lowres, normal quality, ((monochrome)), ((grayscale)), skin spots, acnes, skin blemishes, age spot, glans,extra fingers,fewer fingers, (ugly eyes, deformed iris, deformed pupils, fused lips and teeth:1.2), (un-detailed skin, semi-realistic, cgi, 3d, render, sketch, cartoon, drawing, anime:1.2), text, close up, cropped, out of frame, worst quality, low quality, jpeg artifacts, ugly, duplicate, morbid, mutilated, extra fingers, mutated hands, poorly drawn hands, poorly drawn face, mutation, deformed, blurry, dehydrated, bad anatomy, bad proportions, extra limbs, cloned face, disfigured, gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs, fused fingers, too many fingers, long neck,",
    6: "bad art, blurry, blurry background, bad drawing, poor composition, poor use of textures, poor use of colors, poor art drawing, portrait, ugly, face shot, Overbaked, over contrasted, over saturated, ((3d, cartoon, anime, sketches)), (worst quality:2), (low quality:2), (normal quality:2), lowres, normal quality, ((monochrome)), ((grayscale)), bad anatomy, out of view, cut off, ugly, deformed, mutated, ((young)), EasyNegative, paintings, sketches, (worst quality:2), (low quality:2), (normal quality:2), lowres, normal quality, ((monochrome)), ((grayscale)), skin spots, acnes, skin blemishes, age spot, glans,extra fingers,fewer fingers, (ugly eyes, deformed iris, deformed pupils, fused lips and teeth:1.2), (un-detailed skin, semi-realistic, cgi, 3d, render, sketch, cartoon, drawing, anime:1.2), text, close up, cropped, out of frame, worst quality, low quality, jpeg artifacts, ugly, duplicate, morbid, mutilated, extra fingers, mutated hands, poorly drawn hands, poorly drawn face, mutation, deformed, blurry, dehydrated, bad anatomy, bad proportions, extra limbs, cloned face, disfigured, gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs, fused fingers, too many fingers, long neck,",
    7: "bad art, blurry, blurry background, bad drawing, poor composition, poor use of textures, poor use of colors, poor art drawing, portrait, ugly, face shot, Overbaked, over contrasted, over saturated, ((3d, cartoon, anime, sketches)), (worst quality:2), (low quality:2), (normal quality:2), lowres, normal quality, ((monochrome)), ((grayscale)), bad anatomy, out of view, cut off, ugly, deformed, mutated, ((young)), EasyNegative, paintings, sketches, (worst quality:2), (low quality:2), (normal quality:2), lowres, normal quality, ((monochrome)), ((grayscale)), skin spots, acnes, skin blemishes, age spot, glans,extra fingers,fewer fingers, (ugly eyes, deformed iris, deformed pupils, fused lips and teeth:1.2), (un-detailed skin, semi-realistic, cgi, 3d, render, sketch, cartoon, drawing, anime:1.2), text, close up, cropped, out of frame, worst quality, low quality, jpeg artifacts, ugly, duplicate, morbid, mutilated, extra fingers, mutated hands, poorly drawn hands, poorly drawn face, mutation, deformed, blurry, dehydrated, bad anatomy, bad proportions, extra limbs, cloned face, disfigured, gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs, fused fingers, too many fingers, long neck,",
    8: "bad art, blurry, blurry background, bad drawing, poor composition, poor use of textures, poor use of colors, poor art drawing, portrait, ugly, face shot, Overbaked, over contrasted, over saturated, ((3d, cartoon, anime, sketches)), (worst quality:2), (low quality:2), (normal quality:2), lowres, normal quality, ((monochrome)), ((grayscale)), bad anatomy, out of view, cut off, ugly, deformed, mutated, ((young)), EasyNegative, paintings, sketches, (worst quality:2), (low quality:2), (normal quality:2), lowres, normal quality, ((monochrome)), ((grayscale)), skin spots, acnes, skin blemishes, age spot, glans,extra fingers,fewer fingers, (ugly eyes, deformed iris, deformed pupils, fused lips and teeth:1.2), (un-detailed skin, semi-realistic, cgi, 3d, render, sketch, cartoon, drawing, anime:1.2), text, close up, cropped, out of frame, worst quality, low quality, jpeg artifacts, ugly, duplicate, morbid, mutilated, extra fingers, mutated hands, poorly drawn hands, poorly drawn face, mutation, deformed, blurry, dehydrated, bad anatomy, bad proportions, extra limbs, cloned face, disfigured, gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs, fused fingers, too many fingers, long neck,",
    9: "bad art, blurry, blurry background, bad drawing, poor composition, poor use of textures, poor use of colors, poor art drawing, portrait, ugly, face shot, Overbaked, over contrasted, over saturated, ((3d, cartoon, anime, sketches)), (worst quality:2), (low quality:2), (normal quality:2), lowres, normal quality, ((monochrome)), ((grayscale)), bad anatomy, out of view, cut off, ugly, deformed, mutated, ((young)), EasyNegative, paintings, sketches, (worst quality:2), (low quality:2), (normal quality:2), lowres, normal quality, ((monochrome)), ((grayscale)), skin spots, acnes, skin blemishes, age spot, glans,extra fingers,fewer fingers, (ugly eyes, deformed iris, deformed pupils, fused lips and teeth:1.2), (un-detailed skin, semi-realistic, cgi, 3d, render, sketch, cartoon, drawing, anime:1.2), text, close up, cropped, out of frame, worst quality, low quality, jpeg artifacts, ugly, duplicate, morbid, mutilated, extra fingers, mutated hands, poorly drawn hands, poorly drawn face, mutation, deformed, blurry, dehydrated, bad anatomy, bad proportions, extra limbs, cloned face, disfigured, gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs, fused fingers, too many fingers, long neck,",
    10: "bad art, blurry, blurry background, bad drawing, poor composition, poor use of textures, poor use of colors, poor art drawing, portrait, ugly, face shot, Overbaked, over contrasted, over saturated, ((3d, cartoon, anime, sketches)), (worst quality:2), (low quality:2), (normal quality:2), lowres, normal quality, ((monochrome)), ((grayscale)), bad anatomy, out of view, cut off, ugly, deformed, mutated, ((young)), EasyNegative, paintings, sketches, (worst quality:2), (low quality:2), (normal quality:2), lowres, normal quality, ((monochrome)), ((grayscale)), skin spots, acnes, skin blemishes, age spot, glans,extra fingers,fewer fingers, (ugly eyes, deformed iris, deformed pupils, fused lips and teeth:1.2), (un-detailed skin, semi-realistic, cgi, 3d, render, sketch, cartoon, drawing, anime:1.2), text, close up, cropped, out of frame, worst quality, low quality, jpeg artifacts, ugly, duplicate, morbid, mutilated, extra fingers, mutated hands, poorly drawn hands, poorly drawn face, mutation, deformed, blurry, dehydrated, bad anatomy, bad proportions, extra limbs, cloned face, disfigured, gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs, fused fingers, too many fingers, long neck,",

}

In [ ]:
#@title <left><font face="Monospace" size="4">Run SDXL -> Multi-Prompt</font></left>
working_dir = "/content/ComfyUI-Notebook" #@param{type:"string"}
%cd $working_dir
import gc
import torch
import random
import sdxlrun
from types import SimpleNamespace
from comfy import model_management
from sdxlrun import runsdxl, loadsdxl
from pytorch_lightning import seed_everything
from refinersdxlrun import runsdxlrefiner, loadsdxlrefiner
from iprogress import iprogress

def SDXL_ARGS():
    prompt = "" #@param{type:"string"}
    negativeprompt = "" #@param{type:"string"}
    #@markdown ${\textsf{Base Model Options}}$
    ckpt_path='/content/ComfyUI/models/checkpoints/dreamshaperXL10_alpha2XI10.safetensors'
    ckpt_name='crystalClearxl_ccxl.safetensors' #@param{type:"string"}["crystalClearxl_ccxl.safetensors", "psyanimated_xl.safetensors", "sd_xl_base_1.0_0.9vae.safetensors", "dreamshaperXL10_alpha2XI10.safetensors"]
    lora_name = "None"#@param{type:"string"}["None", "sd_xl_offset_example-lora_1.0.safetensors", "greg_rutkowski_xl_2.safetensors"]
    if lora_name == "None":
        lora_name = None
    else:
        strength_model = 0.5 #@param{type:"number"}
        strength_clip = 0.5 #@param{type:"number"}

    #@markdown ${\textsf{Base Model}}$
    steps = 40 #@param{type:"number"}
    start_step = 0 #@param{type:"number"}
    last_step = 35 #@param{type:"number"}

    #refiner model
    use_refiner =  False #@param{type:"boolean"}
    last_step = last_step if use_refiner else steps
    refiner_steps = steps
    refiner_start_step = last_step
    refiner_last_step = steps

    #@markdown ${\textsf{Image-Batch Settings}}$
    imagewidth = 896 #@param{type:"number"}
    imageheight = 1264   #@param{type:"number"}
    batch_size = 1 #@param{type:"number"}

    cfg = 3 #@param{type:"number"}
    seed = -1 #@param{type:"number"}
    if seed == -1:
        seed = random.randint(1, 432043209)
    seed = seed_everything(seed)

    #@markdown ${\textsf{Sampler Settings}}$
    sampler_name = "dpmpp_2m_sde_gpu" #@param{type:"string"} ["euler", "euler_ancestral", "heun", "dpm_2", "dpm_2_ancestral", "lms", "dpm_fast", "dpm_adaptive", "dpmpp_2s_ancestral", "dpmpp_sde", "dpmpp_sde_gpu", "dpmpp_2m", "dpmpp_2m_sde", "dpmpp_2m_sde_gpu", "ddim", "uni_pc", "uni_pc_bh2"]
    scheduler = "exponential" #@param{type:"string"} ["normal", "karras", "exponential", "simple", "ddim_uniform"]

    #@markdown ${\textsf{Sampler Options}}$
    s_churn = 0. # @param {type:"slider", min:0, max:1, step:0.01}
    s_tmin = 0.1 # @param {type:"slider", min:0, max:1, step:0.01}
    s_tmax = float('inf')
    s_noise = 1. # @param {type:"slider", min:0, max:1, step:0.01}

    eta = 1. #@param{type:"number"}

    order = 4 #@param{type:"slider", min:1, max:4, step:1}

    #@markdown ${\textsf{CLIP Options}}$
    stop_at_last_layer = None #@param{type:"number"}
    width = 4096 #@param{type:"number"}
    height = 4096 #@param{type:"number"}
    crop_w = 0 #@param{type:"number"}
    crop_h = 0 #@param{type:"number"}
    target_width = 4096 #@param{type:"number"}
    target_height = 4096 #@param{type:"number"}

    #@markdown ${\textsf{Noise Options}}$
    denoise = 1.0 #@param{type:"number"}
    force_full_denoise = False #@param{type:"boolean"}
    disable_noise = False #@param{type:"boolean"}
    force_full_denoise = force_full_denoise if use_refiner else True
    noisedevice = "cuda:0" #@param{type:"string"}["cuda:0", "cpu"]

    #@markdown ${\textsf{REFINER OPTIONS}}$
    refinerckpt_name='sd_xl_refiner_1.0_0.9vae.safetensors' #@param{type:"string"} ["sd_xl_refiner_1.0_0.9vae.safetensors", "sd_xl_refiner_1.0.safetensors"]
    refinerforce_full_denoise = True #@param{type:"boolean"}
    refinerdisable_noise = True #@param{type:"boolean"}
    refinerwidth = 4096 #@param{type:"number"}
    refinerheight = 4096 #@param{type:"number"}
    ascore = 3.0 #@param{type:"number"}

    #@markdown ${\textsf{Save Options}}$
    output_folder = "civit" #@param{type:"string"}
    saveprefix = f'{sampler_name}_{scheduler}'
    save_base_image = False #@param{type:"boolean"}

    vae_decode_method = "tiled" #@param{type:"string"}["normal", "tiled"]

    use_preview = True #@param{type:"boolean"}
    disable_metadata = False #@param{type:"boolean"}

    return locals()

sdxl_args = SDXL_ARGS()
sdxl_args = SimpleNamespace(**sdxl_args)
randomize_seed = True #@param{type:"boolean"}

key_count=1
for prompt_id, prompt in prompts.items():
    sdxl_args.prompt = prompt
    sdxl_args.negativeprompt = negative_prompts[key_count]
    print(f'Running with Prompt: {sdxl_args.prompt}')
    print(f'Running with Negative Prompt: {sdxl_args.negativeprompt}')
    if randomize_seed:
        sdxl_args.seed = seed_everything(random.randint(1, 2 ** 64))

    with torch.inference_mode():
        try:
            out
            if sdxl_args.ckpt_name != checkpoint:
                del out
                del model
                del samples
                gc.collect()
                torch.cuda.empty_cache()
                out = loadsdxl(sdxl_args)
                checkpoint = sdxl_args.ckpt_name
            if sdxl_args.use_refiner:
                refined_out
        except:
            out = loadsdxl(sdxl_args)
            checkpoint = sdxl_args.ckpt_name #create checkpoint to trigger loading a new checkpoint
            if sdxl_args.use_refiner:
                refined_out = loadsdxlrefiner(sdxl_args)

        print('Running with settings:')
        print(f'Prompt: {sdxl_args.prompt}')
        print(f'Negative Prompt: {sdxl_args.negativeprompt}')
        for key, value in sdxl_args.__dict__.items():
            if not "prompt" in key and not "negativeprompt" in key:
                print(f'{key}: {value}')
        model, samples = runsdxl(sdxl_args, out)
        model_management.unload_model(model)
        model_management.unload_model(out[1].patcher)
        sdxlrun.get_device_memory()
        if sdxl_args.use_refiner:
            refinermodel, refinedsamples = runsdxlrefiner(sdxl_args, samples, model, refined_out)
            model_management.unload_model(refinermodel)
            model_management.unload_model(refined_out[1].patcher)
            sdxlrun.get_device_memory()
        key_count+=1
    torch.cuda.empty_cache()
    gc.collect()
    checkpoint = sdxl_args.ckpt_name